In [ ]:
import datetime
import _speed_utils as speed_utils
import dask.dataframe as dd
import numpy as np
import geopandas as gpd
import pandas as pd
from segment_speed_utils import gtfs_schedule_wrangling, helpers, segment_calcs
from segment_speed_utils.project_vars import (
    COMPILED_CACHED_VIEWS,
    PROJECT_CRS,
    SEGMENT_GCS,
    #analysis_date,
)
from scripts import A1_sjoin_vp_segments
from shared_utils import calitp_color_palette as cp

In [ ]:
pd.options.display.max_columns = 100
pd.options.display.float_format = "{:.2f}".format
pd.set_option("display.max_rows", None)
pd.set_option("display.max_colwidth", None)

In [ ]:
CONFIG_PATH = './scripts/config.yml'

In [ ]:
STOP_SEG_DICT = helpers.get_parameters(CONFIG_PATH, "stop_segments")

In [ ]:
# Adding analysis_date here since there aren't any files for June yet
analysis_date = '2023-05-17'

### Observations (7/12)
* Step 1: Flagging
    * There are 2,704,812 rows in the dataframe original. About 10% of those rows are flagged as having zeroes in meters elapsed and seconds elapsed. 
    * There are around 4566 routes. About 57% of these routes had at least one trip with one or more rows flagged as zero.
* I took 2 passes at trying to understand why both these columns recorded zeroes.

* Step 2: `vp_pared_stops`. 
    * I grouped `vp_pared_stops` by 'shape_array_key','trip_id', and 'location_timestamp_local' OR `x` and `y`. I counted the number of unique stop sequences after grouping. If this trio had more than one unique stop sequence, that meant the timestamp or location recorded between sequences was duplicated.
    * Only around 9% of rows were flagged as having (obviously) repeated timestamps and locations. 
    * For all of these rows, both the timestamp and location were duplicated. 
    * All the routes that were flagged in step one needed a further look in step 3.
    
* Step 3: `vp_usable`
    * For one route and trip, find: all the recorded vehicle positions, sjoin of vps to segments,
    and the first and last points kept. 
    * Plot the three gdfs in a map to visually inspect what's happening.
    * Compare the sample route and trips with the trip with the highest percentage of non division by 0 rows to see what's going on. 
    
* Buckets of errors (all based on `stage0 vp`). 
    * There is only one recorded point in that segment in the raw data.
        * Fix: use the timestamp that comes after it.
        * <img src= "./speeds_images/only_one_pt_collected.png" width = 300>
    * Points are shared between segments
        * Use p20/p50/p80.
        * <img src= "./speeds_images/shared_vp.png" width = 300>
    * Points recorded are really far out and they don't touch the buffered segments.
        * Fix: figure out % of vehicle positions that are too far out.
        * <img src= "./speeds_images/dots_not_on_seg.png" width = 300>
    * No data captured for that segment at all.
        * Use p20/p50/p80. 
        * <img src= "./speeds_images/no_dots_collected.png" width = 300> 

### Flagging

In [ ]:
# Flagged: all the rows in the df flagged
# divide_by_zero: only the rows that have 0 for meters and sec elapsed
# trips_count: % of trips with 1+ division by 0 row for a route
# route_most_populated_df: the trip for a route with the smallest % of rows that are divided by 0
flagged, divide_by_zero, trips_count, route_most_populated_df = speed_utils.flagging_stage(analysis_date)

In [ ]:
divide_by_zero.loop_or_inlining.value_counts()

### Fixes

#### VPs too far out 

In [ ]:
STG_0_FILE = STOP_SEG_DICT['stage0']
    
all_pts = helpers.import_vehicle_positions(
            SEGMENT_GCS,
            f"{STG_0_FILE}_{analysis_date}/",
            "df",
            columns = ["gtfs_dataset_key", "trip_id","geometry"],
            partitioned = False
        )
all_pts = all_pts.compute()

In [ ]:
STG_1_FILE = STOP_SEG_DICT['stage1']
unique_trips = A1_sjoin_vp_segments.add_grouping_col_to_vp(
        f"{STG_1_FILE}_{analysis_date}",
        analysis_date,
       ["shape_array_key"]
    )

In [ ]:
m1 = all_pts.merge(unique_trips, on = ["gtfs_dataset_key", "trip_id"],
            how = "inner"
        )

In [ ]:
m1.head()

### Investigate 

In [ ]:
stage3_df = speed_utils.flag_stage3(divide_by_zero, analysis_date)

In [ ]:
stage3_df.shape_array_key.nunique()

In [ ]:
sort_by = ['_gtfs_dataset_name','shape_array_key','trip_id','stop_sequence']

In [ ]:
preview_cols = sort_by + ['stop_id','gtfs_dataset_key','location_timestamp_local','pair','stage3_flag']

#### Find routes with a lot of n_trips that need to be evaluated.

In [ ]:
shape_trips = speed_utils.count_trips_routes(stage3_df)

In [ ]:
shape_trips.head(5)

#### Example Trip 1
* Understanding the result from flag_stage3().
* Looking at AC Transit: stop sequences 6 and 7 have different stop_ids. However, their time stamps and locations are the same.
* It looks like they share the same point.
* However, this isn't due to paring too many points: there just aren't enough points to choose from in the raw data.
* Also sequence 2 is extremely long.

In [ ]:
stage3_df[(stage3_df.stage3_flag != 'check in stage 2')].sort_values(by =sort_by)[preview_cols].head(2)

In [ ]:
test_route1 = "03bc2779a66739316156b459ffc3eefa"
test_gtfs_key1 = "cdd2ad81863b6d4ad51676a1cb781ea8"
test_trip1 = "11776020"

In [ ]:
# % of trips with problematic rows for this route
trips_count[trips_count.shape_array_key == test_route1]

In [ ]:
speed_utils.original_df_rows(flagged, test_trip1, test_route1)

In [ ]:
speed_utils.stage2_trouble_shooting(flagged_df= stage3_df,
                        date = analysis_date,
                        route = test_route1,
                        trip = test_trip1,
                        gtfs_key = test_gtfs_key1)

#### Example Trip 2
* Underestanding the result from `flag_stage3()`. 
* Same thing as Ex Trip 1 except kooking at LA Metro: stop sequences 45 and 46 have different stop_ids. However, their time stamps and locations are the same. 
* Same issue: segments 45 and 46 are sharing points. There aren't enough points captured.
* In general, it looks like this route doesn't have a lot of rows that are ok.

In [ ]:
test_route2 = "38c814829dff816aa87c606c3aab4f45"
test_gtfs_key2 = "65d9589130415c685b89f4f7c2d8bd7e"
test_trip2 = "10294000051654-DEC22"

In [ ]:
# Original number of rows for this trip
len(flagged[(flagged.trip_id == test_trip2) & (flagged.shape_array_key == test_route2)])

In [ ]:
# Rows with zeroes...a lot of them.
len(divide_by_zero[(divide_by_zero.trip_id == test_trip2) & (divide_by_zero.shape_array_key == test_route2)])

In [ ]:
stage3_df[(stage3_df.stage3_flag != 'check in stage 2') & (stage3_df.shape_array_key == test_route2) 
   & (stage3_df.stop_sequence.isin([45,46]))].sort_values(by =sort_by)[preview_cols]

In [ ]:
# % of trips with problematic rows for this route, lots of them
trips_count[trips_count.shape_array_key == test_route2]

In [ ]:
speed_utils.original_df_rows(m1, test_trip2, test_route2)

In [ ]:
speed_utils.stage2_trouble_shooting(flagged_df= stage3_df,
                        date = analysis_date,
                        route = test_route2,
                        trip = test_trip2,
                        gtfs_key = test_gtfs_key2,)

In [ ]:
# See which trip has the most rows that are ok for this route
route_most_populated_df[route_most_populated_df.shape_array_key == test_route2]

In [ ]:
# Plot the trip from this route with the highest % of ok rows
speed_utils.stage2_trouble_shooting(flagged_df= stage3_df,
                        date = analysis_date,
                        route = test_route2,
                        trip = "10294000051724-DEC22",
                        gtfs_key = test_gtfs_key2,)

#### Example Trip 3
* Choosing a route/trip with high n_trips that isn't Muni or LA Metro to shake things up.
* San Diego Vehicle Positions
* Segments 44 and 45 don't have any rows in the original dataframe for trip 16938440.

In [ ]:
test_route3 = "1fc55d9df0cd785dddc864bf1b72976f"
test_gtfs_key3 = "a4f6fd5552107e05fe9743ac7cce2c55"
test_trip3 = "16938440"

In [ ]:
speed_utils.original_df_rows(flagged, test_trip3, test_route3)

In [ ]:
speed_utils.stage2_trouble_shooting(flagged_df= stage3_df,
                        date = analysis_date,
                        route = test_route3,
                        trip = test_trip3,
                        gtfs_key = test_gtfs_key3,)

In [ ]:
# Trip from this route with the most ok rows
route_most_populated_df[route_most_populated_df.shape_array_key == test_route3]

In [ ]:
# Test the trip with the most ok rows
speed_utils.stage2_trouble_shooting(flagged_df= stage3_df,
                        date = analysis_date,
                        route = test_route3,
                        trip = "16938341",
                        gtfs_key = test_gtfs_key3,)

In [ ]:
speed_utils.original_df_rows(flagged, "16938341", test_route3)